In [1]:
import numpy as np
from collections import Counter

In [2]:
class Node:
    def __init__(self, ids=None, children=[], entropy=0, depth=0):
        self.ids = ids           # index of data in this node
        self.entropy = entropy   # entropy, will fill later
        self.depth = depth       # distance to root node
        self.split_attribute = None # which attribute is chosen, it non-leaf
        self.children = children # list of its child nodes
        self.order = None       # order of values of split_attribute in children
        self.label = None       # label of node if it is a leaf

    def set_properties(self, split_attribute, threshold):
        self.split_attribute = split_attribute
        self.threshold = threshold

    def set_label(self, label):
        self.label = label

In [3]:
class Tree:
    def __init__(self, max_depth=10, min_samples_split=2, min_gain=1e-4):
        self.root = None
        self.max_depth = max_depth 
        self.min_samples_split = min_samples_split
        self.Ntrain = 0
        self.min_gain = min_gain
    

    def fit(self, X, y):
        self.Ntrain = len(X)
        self.X = X
        self.attributes = [i for i in range(X[0].shape[0])]
        self.y = y
        self.labels = np.unique(self.y)
        
        ids = range(self.Ntrain)
        self.root = Node(ids=ids, entropy=self._entropy(ids), depth=0)
        queue = [self.root]
        while queue:
            node = queue.pop()
            if node.depth < self.max_depth or node.entropy < self.min_gain:
                node.children = self._split(node)
                if not node.children: #leaf node
                    self._set_label(node)
                queue += node.children
            else:
                self._set_label(node)
                
    def _entropy(self, ids):
        # calculate entropy of a node with index ids
        if len(ids) == 0: return 0
        #ids = [i+1 for i in ids] # panda series index starts from 1
        #freq = np.array(self.y[ids].value_counts())
        uni, cnt = np.unique(self.y[ids], return_counts="true")
        
        cnt = cnt[cnt != 0]
        ratio = cnt / np.sum(cnt)
        return -np.sum(ratio*np.log(ratio))

    def _set_label(self, node):
        # find label for a node if it is a leaf
        # simply chose by major voting 
        #y_ids = [i + 1 for i in node.ids]  # y is a series variable
        #node.set_label(self.y[y_ids].mode()[0]) # most frequent label
        uni, cnt = np.unique(self.y[node.ids], return_counts="true")
        print(uni, cnt)
        most_common = Counter(dict(zip(uni, cnt))).most_common()[0][0]
        node.set_label(most_common)
    

    def _split(self, node):
        ids = node.ids 
        best_gain = 0
        best_splits = []
        best_attribute = None
        best_threshold = None
        node_X = self.X[ids]
        
        for att in self.attributes:
            best_HxS = 1000
            splits = []
            threshold = 0
            
            for val in np.unique(node_X[:, att]):
                threshold = val
                splits.append([i for i, x in enumerate(node_X[:, att]) if x <= threshold])
                splits.append([i for i, x in enumerate(node_X[:, att]) if x > threshold])
                #splits = np.vstack((lesser, greater))
                if min(map(len, splits)) < self.min_samples_split: continue
                HxS = 0
                for split in splits:
                    HxS += len(split)*self._entropy(split) / len(ids)
                
                if HxS < best_HxS:
                    best_HxS = HxS
                    
            gain = node.entropy - best_HxS
            if gain < self.min_gain: continue # stop if small gain 
            if gain > best_gain:
                best_gain = gain 
                best_splits = splits
                best_attribute = att
                best_threshold = threshold
                
        node.set_properties(best_attribute, best_threshold)
        child_nodes = [Node(ids=split,
                       entropy=self._entropy(split), depth=node.depth+1) for split in best_splits]
        
        return child_nodes 
    
    def predict(self, X):
        n = len(X)
        predict = []
        
        for i in range(n):
            node = self.root
            while node.children:
                if x[i][node.split_attribute] <= node.threshold:
                    node = node.children[0]
                else:
                    node = node.children[1]
            predict.append(node.label)
            
        return predict
    
    def score(self, predict, y):
        counter = 0
        for i in range(len(y)):
            if predicted[i] == y[i]:
               ++counter 
            
        return counter*100/len(y)
                

In [4]:
data = np.loadtxt("diabetes.csv", delimiter=",")
data = data[:100]
X = data[:, :-1]
y = data[:, -1]
id3 = Tree(max_depth=4, min_samples_split=2)
id3.fit(X, y)
id3.predict(X[0])

[] []


IndexError: list index out of range